In [229]:
#Notes:
#1) Instead of pre-processing some data, see if someone has already processed it
#2) Instead of doing task X (which you've never done before), see if someone has already done it
#3) I really want to make a decorator for python s.t. I can just create progress bars by writing @bar before a for loop


In [4]:
from bs4 import BeautifulSoup
import requests
import re


In [28]:
#Attempt 1: This was a SLOW and inelegant way to do things, as the wandering inn wiki has an (exhaustive?) list of [Classes] and
#[Skills]
#Note that when I did this, I messed up the scraping a bit and duplicated some of the data. So you might not get a superset
#of my data if you try running this notebook.

URL = "https://wanderinginn.com/table-of-contents/"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")


chapter_links = []
for link in soup.find_all('a'):
    chapter_links.append(link.get('href'))
chapter_links = chapter_links[27:-16]

chapter_n = requests.get(chapter_links[1])

soup_chapter_n = BeautifulSoup(chapter_n.content, "html.parser")
chapter_n_text = soup_chapter_n.find("div", {"class", "entry-content"}).get_text()
chapter_n_text


x = re.findall("[\[]{1}.*?[\]{1}]", "[Deathbolt] spell of the [Necromancer]")
assert x == ["[Deathbolt]", "[Necromancer]"]
get_skills_and_classes = lambda chapter_text: re.findall("[\[]{1}.*?[\]{1}]", chapter_text)
assert get_skills_and_classes(chapter_n_text)[0] == '[Innkeeper Class Obtained!]'

#Uncomment when you run
NN_text = []

#I had to iterate through this manually, running the code and incrementing last_n by 30 each time. Why? Because it was taking too
#long to just iterate through range(len(chapter_links)) and I was worried that there might be anti scraping measures or something
#that prevent me from doing it all repeatedly or whatever. That's probably dumb though.
#To run it yourself, set n = 0 and keep going till you hit the largest N s.t. 30 N < len(chapter_links)
#Or just change range(30) to range(len(chapter_links)) and get rid of the if clause

last_n = 0
for n in range(len(chapter_links)):
    if n+last_n>613:
        break
    chapter_n = requests.get(chapter_links[n+last_n])

    soup_chapter_n = BeautifulSoup(chapter_n.content, "html.parser")
    chapter_n_text = soup_chapter_n.find("div", {"class", "entry-content"}).get_text()
#Uncomment when you run
    NN_text += list(set(get_skills_and_classes(chapter_n_text)))

NN_text[-300:]

#Dataset has some announcements and I was too lazy to find whether they had special tags accompnying them in the soup, so I just
#made a hacky regex that probably won't remove all the announcements
x = re.findall("[\[]{1}([a-zA-Z]|[,])*[\]{1}]", "[Deathbolt] spell of the [Necromancer]")
is_annoucement = lambda txt: 1 if re.search("another interview|.* [aA]uthor is on break.*|.* monthly break.*|.*I am forgetting.*|.*conducting a survey.*|.*[eE]\-book.*|.*Merch.Store.*|.*Author.has.*|.*is now available for.*|.*is out now.*|.*now has a.*|.*filled out the.|.*[Pp]atreon.*|.*has released.*", txt) !=None else 0
NN_no_announcements = []
for i in range(len(NN_text)):
    if not is_annoucement(NN_text[i]):
        NN_no_announcements.append(NN_text[i])
#Unless something has gone wrong, TWI should have a couple of things which match is_announcement and so NN_no_announcements
#should be a strict sublist of NN_text
assert len(NN_no_announcements) <= len(NN_text)

is_def_skill = lambda txt: 1 if re.search(".*[sS]kill.*",txt)!=None  else 0

assert is_def_skill('[Undying Lich, Myth of Death and Vengeance Level 78!]') == 0
assert is_def_skill("[Necromancer]")==False

NN_def_skills = []
for i in NN_no_announcements:
    if is_def_skill(i):
        NN_def_skills.append(i)


KeyboardInterrupt: 

28574

In [173]:
is_skill_change("[Conditions Met: Kicker → Football Player Class]")

1

In [157]:
#Attempt 2: Way less complicated
#Lesson learnt: Why clean the data if someone else has already done it for you?
page_skill = requests.get("https://thewanderinginn.fandom.com/wiki/Skills")
soup_skill = BeautifulSoup(page_skill.content, "html.parser")
page_class = requests.get("https://thewanderinginn.fandom.com/wiki/Classes")
soup_class= BeautifulSoup(page_class.content, "html.parser")
page_spell = requests.get("https://thewanderinginn.fandom.com/wiki/Spells")
soup_spell= BeautifulSoup(page_spell.content, "html.parser")


skills = []
classes = []
spells = []
soup_skill_tables = soup_skill.find_all("table", {"class", "wikitable"})
is_skill = lambda txt: 1 if re.search("^\[.*\]\n.*", txt) !=None and re.search(".*\[[0-9]*\].*",txt) ==None else 0
soup_skill_td = soup_skill.find_all("td")
for i in soup_skill_td:
    if is_skill(i.get_text()):
        skills.append(i.get_text())


soup_class_tables = soup_class.find_all("table", {"class", "wikitable"})
is_class = lambda txt: 1 if re.search("^\[.*\]\n.*", txt) !=None and re.search(".*\[[0-9]*\].*",txt) ==None else 0
soup_class_td = soup_class.find_all("td")
for i in soup_class_td:
    if is_class(i.get_text()):
        classes.append(i.get_text())

is_spell = lambda txt: 1 if re.search("^\[.*\]\n.*", txt) !=None and re.search(".*\[[0-9]*\].*",txt) ==None else 0

def turn_into_spell(txt):
    
    if txt[0]!="[":
        txt = "["+txt
    if txt[-1]!="]":
        txt+="]"
    return txt

soup_spell_td = soup_spell.find_all("td")
for i in soup_spell_td:
    if is_spell(i.get_text()):
        i_txt = i.get_text()
        sep_i = i_txt.split("][")
        if len(sep_i) ==1:
            spells+=sep_i
        else:
            sep_i = [turn_into_spell(j) for j in sep_i]
            spells+=sep_i
            
spells

['[Cooling Breeze]\n',
 '[Flashbang]\n',
 '[Flashlight]\n',
 '[Grow Grass]\n',
 '[Hot Hands]\n',
 '[Light]\n',
 '[Noise]\n',
 '[Spark]\n',
 '[Thorn Hand]\n',
 '[Water Spray]\n',
 '[Cooling Wind]\n',
 '[Detect Death]\n',
 '[Electric Jolt]\n',
 '[Everdawn’s Radiance]\n',
 '[Frozen Wind]',
 '[Frozen Winds]\n]',
 '[Glitterfrost Gust]\n',
 '[Light Arrow]\n',
 '[Water Jet]\n',
 '[Call Object]\n',
 '[Create Snow]\n',
 '[Flame Bolt]',
 '[Fire Bolt]',
 '[Firebolt]\n]',
 '[Flame Spray]\n',
 '[Flash]\n',
 '[Ice Floor]',
 '[Icy Floor]',
 '[Frozen Floor]\n]',
 '[Ice Shard]',
 '[Icy Shard]\n]',
 '[Illumination]\n',
 '[Jet Spray]\n',
 '[Light Sconce]\n',
 '[Lightning Jolt]\n',
 '[Lightning Orb]\n',
 '[Mud Slap]\n',
 '[Shock Volt]\n',
 '[Snowball]\n',
 '[Steel Arrows]\n',
 '[Sticky Webs]\n',
 '[Tripgrass]\n',
 '[Arrows of Sand]\n',
 '[Detect Magic]\n',
 '[Fire Orbs]\n',
 '[Flaming Swathe]\n',
 '[Flash Hooves]\n',
 '[Fox Fire]\n',
 '[Galewinds] / [Gale Winds]\n',
 '[Icy Spear]\n',
 '[Karas Duststorm]\n

In [187]:
skill_set = set([i.strip("\n") for i in skills])
spell_set = set([i.strip("\n") for i in spells])


{'',
 '[Slayer]',
 '[Fartraveller Bard]',
 '[King of Intrigue]',
 '[ Bloodglass Mage]',
 '[Boosted Morale (Unit)]',
 '[Great Chieftain ]',
 '[Armor Knights]',
 '[Method Acting]',
 '[Peerless Spearmaster]',
 '[Bone Claws]',
 '[Long Backstep]',
 '[Corruptor]',
 '[Ransomer]',
 '[Cure Ailments]',
 '[Great General]',
 '[Come to the Table]',
 '[Lesser Stamina]',
 '[Ram’s Charge]',
 '[Wild Gallop]',
 '[Shipbuilder]',
 '[Glassblower]',
 '[Bone Fracture]',
 '[Restore]',
 '[Arc Arrows]',
 '[No Retreat No Surrender Hold Your Ground]',
 '[Hurricane Stabs]',
 '[Naq-Alrama Smith]',
 '[Echoing Strings]',
 '[City Crier]',
 '[Reinforced Block]',
 '[Pavilion of Secrets]',
 '[Even Footing]',
 '[Overpowering Blow]',
 '[Swift Infantry]',
 '[Marked Target]',
 '[Historian of the World ]',
 '[Woodcutters]',
 '[Cleansed Materials]',
 '[March of the Invincible]',
 '[Swift Slash]',
 '[Noblewomen]',
 '[Bone Surgeon]',
 '[Lead Engineer]',
 '[Bull’s Charge]',
 '[Slick Spell: Ice Floor]',
 '[Halberdier]',
 '[He Scra

In [342]:
class_set.issuperset(set(["[Lesser Dexterity]"]))

False

In [231]:

is_change = lambda txt: 1 if re.search(".*→.*",txt)!=None else 0
clean_txt = lambda pattern, repl, txt: re.sub(pattern, repl, txt)

#Note that I had to get rid of stuff like [Conditions Met: Goth → Abyssal W—] or else I'd have been there for a while
def get_NN_clean(NN_no_announcements):
    NN_split = []
    for i in NN_no_announcements:
        if is_change(i):
            x = i.split("→")
            x = [turn_into_spell(j) for j in x]
            NN_split+= x
        else:
            NN_split.append(i)
    
    NN_clean = []
    for i in NN_split:
        x = clean_txt("...+\[.*\]|.* Users .*[Cc]onnected.|.* has disconnected.*|.*joined the conversation.*|.*[a-zA-Z]+[0-9]+.*|.*[0-9]+[a-zA-Z]+.*|.*[a-zA-Z]+_[0-9]+.*|.*batman.|\[.\]", "", i)
        x = clean_txt(">.*", "\]", x)
        if x!="[]" and x!="" :
            NN_clean.append(turn_into_spell(x))
    return NN_clean
NN_clean = get_NN_clean(NN_no_announcements)
set(NN_clean)
len(NN_clean)

28601

In [357]:
clean_txt("[\—\-\–]", " ", "[—–]")

'[  ]'

In [536]:
".*[Cc]onditions [Mm]et :|[a-zA-Z]*[0-9]+|^Skill -|Ob[—-]|Skill – |Ski[-—]||Level [0-9]!*|Level[sing]*| Learned.|Ob[-—]|Class | acquired |[Oo]btained!|"
def get_potential_NN_classes(NN_clean, spells_set, skills_set):
    temp = []
    for i in NN_clean:
        x = clean_txt("|Ritual|[Ll]esser [Bb]ond|learned|Lesser Bond|removed|Weapon Art|Artform|Miracle [\—\-\–]|siphoned|Empowered Spell: |\.|Aura Binding:|(Bind|Bound) Spell:|(temporary)|granted.|received|Legacy [\—\-\–] |acquired|!|lost.|Spell [\—\-\–]* |Temporary Skill Assigned:|Temporary Skill|Consolidation Failed. Unable to Advance|Condition [\—\-\–]|[Cc]onditions* [Mm]et:|[a-zA-Z]*[0-9]+|^Skill -|Ob[\—\-\–]|Skill [\—\-\–] |Ski[\—\-\–]|Level [0-9]!*|Level|Level[sing]*| Learned.|Ob[\—\-\–]|Class | acquired |[Oo]btained!|Consolidation:|.*~.*~.*|\\{2}|Condition [\—\-\–]|[Cc]onditions* [Mm]et:|[a-zA-Z]*[0-9]+|^Skill -|Ob[\—\-\–]|Skill [\—\-\–] |Ski[\—\-\–]|Level [0-9]!*|Level|Level[sing]*| Learned.|Ob[\—\-\–]|Class | acquired |[Oo]btained!*| !", "", i)
        x = clean_txt("\[ ", "\[",x)
        x = clean_txt(" \]", "\]",x)
        x = x.replace("\\","")
        temp.append(x)
        
    temp = set(temp)
    temp = temp.difference(spells_set)
    temp = temp.difference(skills_set)
    
    return temp
class_set = get_potential_NN_classes(NN_clean, spell_set, skill_set)
class_txt = "\n".join([i for i in list(class_set)])

In [539]:
with open("classes_cleaned.txt", "w", encoding="utf-8") as f:
    f.write(class_txt)

In [546]:
with open("classes_cleaned.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
#temp = lambda txt: clean_txt("!| [Oo]btained|Level |levelled|[0-9]+|.*[\—\-\–].*|.*[Ss]kill.*" ,"", txt)
def temp_1(lines):
    temp = lambda txt: clean_txt(".*Rank  Horror.*|Temporary|.*:.*|[Rr]emoved|Class|.*Boon of the.*|.*\(.*|Aspect.*|\.|My Noble Virtue:.*|Bound Spell:.*|Class Consolidation Failed. Unable to Advance|^\[\\n$|^\[\].*|^\[$|Class Consolidation: |Spell [\—\-\–].*|Condition.*|Conditions Met.*|.*Miracle.*|!| [Oo]btained|Level |levelled|[0-9]+|.*[\—\-\–]{1}\]|.*[Ss]kill.*" ,"", txt)
    remove_extra_spaces = lambda txt: clean_txt("\[ ","[" ,clean_txt(" \]", "]", txt))
    x = [];
    for i in range(len(lines)):
        line = lines[i]
        if temp(line) != "\n":
            x.append(remove_extra_spaces(temp(line)))
    return x
new_lines = list(map(lambda x: x+"\n",  list(set(map( lambda x: x.strip("\n"), temp_1(lines))).difference(spell_set).difference(skill_set))))
with open("classes_cleaned.txt", "w", encoding="UTF-8") as f:
    f.write("".join(new_lines))
f.close()

In [547]:
len(lines)

3312

In [512]:
def apply_pred(ls, pred):
    x=[]
    for i in ls:
        if pred(i):
            x.append(i)
    return x

3186

In [525]:
len(set(map( lambda x: x.strip("\n"), temp_1(lines))))

3201